In [1]:
%pip install oracledb


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:

customer_dim_insert = ''' insert into customer_dim (CustomerId, FirstName, LastName, Email, Phone, Address, MembershipDate) values(:1, :2, :3, :4, :5, :6, :7) '''


In [13]:
import oracledb
import pandas as pd

# Connect to the Oracle database
operational_connection = oracledb.connect(user="walmart", password="walmart", dsn="127.0.0.1:1521/freepdb1")
print("Connected to Oracle Database")
 
# Create a cursor to execute queries
operational_cursor = operational_connection.cursor()
def get_data_frame(cursor, query):
    # Execute the SQL query
    cursor.execute(query)
    # Fetch all rows from the query result
    data = cursor.fetchall()
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]
    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

Connected to Oracle Database


In [17]:
customer_df = get_data_frame(operational_cursor, "select CustomerId, FirstName, LastName, Email, Phone, Address, MembershipDate from Customers")
customer_df

,CUSTOMERID,FIRSTNAME,LASTNAME,EMAIL,PHONE,ADDRESS,MEMBERSHIPDATE
0,86,Alexander,Peterson,alexander.peterson@gmail.com,555-186-6789,"8600 Birchwood Lane, Seattle, WA, USA",2021-10-12
1,87,Mia,Cook,mia.cook@yahoo.com,555-187-7890,"8700 Palm Avenue, Portland, OR, USA",2022-03-16
2,88,Lucas,Bailey,lucas.bailey@hotmail.com,555-188-8901,"8800 Elm Street, Denver, CO, USA",2023-06-29
3,89,Amelia,Gray,amelia.gray@outlook.com,555-189-9012,"8900 Spruce Avenue, Columbus, OH, USA",2021-12-07
4,90,Logan,Powell,logan.powell@aol.com,555-190-0123,"9000 Redwood Boulevard, Kansas City, MO, USA",2022-05-01
...,...,...,...,...,...,...,...
95,81,Olivia,Reed,olivia.reed@gmail.com,555-181-1234,"8100 Maplewood Drive, Buffalo, NY, USA",2023-02-10
96,82,Henry,Brooks,henry.brooks@yahoo.com,555-182-2345,"8200 Oak Ridge, Richmond, VA, USA",2021-09-25
97,83,Ava,Kelly,ava.kelly@hotmail.com,555-183-3456,"8300 Willow Park, Atlanta, GA, USA",2022-07-18
98,84,Jack,Ward,jack.ward@outlook.com,555-184-4567,"8400 Pinecrest Street, Minneapolis, MN, USA",2023-11-01


In [19]:
customer_df = get_data_frame(operational_cursor, "select CustomerId, FirstName, LastName, Email, Phone, Address, MembershipDate from Customers")
# Connect to the Oracle database
star_connection = oracledb.connect(user="walmart_star_schema", password="walmart", dsn="127.0.0.1:1521/freepdb1")
star_cursor = star_connection.cursor()
print("Connected to Oracle Database")
def insert_data_frame(cursor, query, dataFrame):
    for i, row in dataFrame.iterrows():
        cursor.execute(query, tuple(row))
    cursor.execute('commit')

Connected to Oracle Database


In [20]:
insert_data_frame(star_cursor, customer_dim_insert ,customer_df)

In [21]:
customer_df

,CUSTOMERID,FIRSTNAME,LASTNAME,EMAIL,PHONE,ADDRESS,MEMBERSHIPDATE
0,86,Alexander,Peterson,alexander.peterson@gmail.com,555-186-6789,"8600 Birchwood Lane, Seattle, WA, USA",2021-10-12
1,87,Mia,Cook,mia.cook@yahoo.com,555-187-7890,"8700 Palm Avenue, Portland, OR, USA",2022-03-16
2,88,Lucas,Bailey,lucas.bailey@hotmail.com,555-188-8901,"8800 Elm Street, Denver, CO, USA",2023-06-29
3,89,Amelia,Gray,amelia.gray@outlook.com,555-189-9012,"8900 Spruce Avenue, Columbus, OH, USA",2021-12-07
4,90,Logan,Powell,logan.powell@aol.com,555-190-0123,"9000 Redwood Boulevard, Kansas City, MO, USA",2022-05-01
...,...,...,...,...,...,...,...
95,81,Olivia,Reed,olivia.reed@gmail.com,555-181-1234,"8100 Maplewood Drive, Buffalo, NY, USA",2023-02-10
96,82,Henry,Brooks,henry.brooks@yahoo.com,555-182-2345,"8200 Oak Ridge, Richmond, VA, USA",2021-09-25
97,83,Ava,Kelly,ava.kelly@hotmail.com,555-183-3456,"8300 Willow Park, Atlanta, GA, USA",2022-07-18
98,84,Jack,Ward,jack.ward@outlook.com,555-184-4567,"8400 Pinecrest Street, Minneapolis, MN, USA",2023-11-01
